In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from functions import loading, plot_heatmap, plot_linouts, plot_all,add_har_order, cmap_map, Alps1, plot_har_yield
import os
from scipy import ndimage
import pandas as pd
from matplotlib import cycler
from matplotlib.colors import Normalize, LogNorm, TwoSlopeNorm, PowerNorm, FuncNorm
from matplotlib import colorbar
from matplotlib.ticker import MultipleLocator
from mycolorpy import colorlist as mcp
from HARP.units import Unit as u
light_cmap = plt.get_cmap('magma')#cmap_map(lambda x: x/2 + 0.5, plt.get_cmap('magma'))
from scipy.signal import savgol_filter, butter, find_peaks
def cm_to_inch(x):
    return x/2.54
from itertools import product
plt.rcParams.update({'axes.labelsize':18, 'image.cmap': light_cmap, 'font.family':'Arial', "figure.figsize":(cm_to_inch(18),cm_to_inch(11.5)), 'font.size':18})



In [ ]:
save_folder_art = '/Volumes/KasiaDrive/Processed_Artemis'
filesart = [i for i in os.listdir(save_folder_art) if 'MgO' and 'part2' in i]
filesart.sort()
filesart

## Testing Rotations Scans


In [ ]:
rot800 = loading(save_folder_art, "Scan_220620_('MgO200',)um_800longnm_part2_test_theta_multi_70000MCP.npz")

In [ ]:
rot800

In [ ]:
fig, ax  = plt.subplots(1, figsize=(15,15))
plot_all(rot800, 'rotation', (3.3*1240/780,14), (0,-1), 780, fig = fig, ax=ax, dp=5)
plt.show()

In [ ]:
scan = rot800.copy()
#scan.data = scan.data.apply(lambda x: x - min(x.flatten()))
#scan.data = scan.data.apply(lambda x: savgol_filter(x,11,3))

In [ ]:
fig, ax  = plt.subplots(1, figsize=(15,15))
plot_all(scan, 'rotation', (3.3*1240/780,14), (250,350), 780, fig = fig, ax=ax, dp=10, norm = LogNorm())
plt.show()

In [ ]:
fig, ax  = plt.subplots(1)
ev, deg, Z = plot_heatmap(scan, 'rotation', (3.3*1240/780,12), (250,350), 780, fig = fig, ax=ax, dp=10, norm = LogNorm(vmax=2e2))
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,10), subplot_kw={'projection': 'polar'})


cmap = plt.get_cmap('BuPu')
r, theta = np.meshgrid(ev, np.radians(deg))
im = ax[0].contourf(theta,r, Z, cmap='BuPu', levels=50, vmin=120)
# ax[0].set_thetalim(np.radians(0), np.radians(180))
ax[0].set_rlim(5,9)
ax[0].set_rticks([6,8])
ax[0].set_xlabel('Energy [eV]')

im2 = ax[1].contourf(theta,r, Z, cmap='BuPu', vmax=120,levels=100)
#ax[1].set_thetalim(np.radians(0), np.radians(180))
ax[1].set_rlim(8,12)
ax[1].set_rticks([8,10,12])
ax[1].set_xlabel('Energy [eV]')


## Intensity Focus Scans


In [ ]:
int800 = loading(save_folder_art, "Scan_220620_('MgO200',)um_800longnm_part2_intensityfocusscan_70000MCP.npz")

In [ ]:
scan = int800.copy()
scan.iris.unique()

### Plotting traces only

In [ ]:
for iris in [82,83,84,85]:
    fig, ax = plt.subplots(1, figsize=(15,8))
    stage='sampleX'
    countbig = 0
    scan = int800.copy()
    scan = int800.copy()[scan.iris == iris]
    N = len(scan[stage].unique())
    shape = scan.data.values[0].shape
    print(shape)
    Z = np.zeros((N, shape[0], shape[1]))
    Y = np.zeros(N)
    count = 0
    for i in np.arange(N)[scan[stage].unique().argsort()]:
        Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()
        Y[count] = scan[stage].unique()[i]
        count+=1
    print(Z.shape)
    y = np.arange(len(Y)*(shape[0]))
    im = ax.imshow(Z.reshape(31*(shape[0]),shape[1]), aspect = 'auto', norm=LogNorm(vmax=1000, vmin=100), cmap='ocean_r')
    ax.set_title(('WP position', iris))
    ax.set_yticks(y[shape[0]//2::shape[0]])
    ax.set_yticklabels(Y)
    plt.colorbar(im, ax=ax)
    plt.xlim(0,1200)
    plt.show()

### Tracing Harmonics

In [ ]:
for iris in [82,83,84,85]:
    fig, ax = plt.subplots(1, figsize=(15,8))
    stage='sampleX'
    countbig = 0
    scan = int800.copy()
    scan = int800.copy()[scan.iris == iris]
    N = len(scan[stage].unique())
    shape = scan.data.values[0].shape
    print(shape)
    Z = np.zeros((N, shape[0], shape[1]))
    Y = np.zeros(N)
    count = 0
    for i in np.arange(N)[scan[stage].unique().argsort()]:
        Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()
        Y[count] = scan[stage].unique()[i]
        count+=1
    y = np.arange(len(Y)*shape[0])
    im = ax.imshow(Z.reshape(31*shape[0],shape[1]), aspect = 'auto', norm=LogNorm(vmax=1000, vmin=100), cmap='ocean_r')
    ax.axvspan(20,120, alpha=0.2, color='red')
    ax.axvspan(160,290, alpha=0.2, color='blue')
    ax.axvspan(450,550, alpha=0.2, color='white')
    ax.axvspan(550,650, alpha=0.2, color='yellow')
    ax.axvspan(800,950, alpha=0.2, color='orange')
    ax.axvspan(1020,1100, alpha=0.2, color='purple')
    ax.axvspan(1120,1200, alpha=0.2, color='black')

    ax.set_title(('WP position', iris))
    ax.set_yticks(y[shape[0]//2::shape[0]])
    ax.set_yticklabels(Y)
    plt.colorbar(im, ax=ax)
    plt.xlim(0,1300)
    plt.show()
    

hlim = [(20,120), (160,290), (450,550), (550,650), (800,950), (1020,1100), (1120,1200)]
h_color = ['Red', 'Blue (5th?)', 'White', 'Yellow (7th?)', 'Orange (9th?)', 'Pink (11th?)', 'Black']

In [ ]:
for iris in [82, 83, 84, 85]:
    fig, ax = plt.subplots(1, figsize=(15,8))
    stage='sampleX'
    countbig = 0
    vspan = hlim[0]
    scan = int800.copy()
    scan = int800.copy()[scan.iris == iris]
    N = len(scan[stage].unique())
    shape = scan.data.values[0].shape
    print(shape)
    Z = np.zeros((N, shape[0],vspan[1]-vspan[0]))
    Y = np.zeros(N)
    count = 0
    for i in np.arange(N)[scan[stage].unique().argsort()]:
        Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()[:,vspan[0]:vspan[1]]
        Y[count] = scan[stage].unique()[i]
        count+=1

    ax.axes.set_prop_cycle(cycler(color = mcp.gen_color(cmap="plasma",n=N+1)))
    for i in range(Z.shape[0]):
        ax.plot(savgol_filter(Z.mean(axis=1)[i], 21, 3) - min(savgol_filter(Z.mean(axis=1)[i], 21, 3)), label=Y[i])

    ax.set_title((h_color[0],'WP position', iris, ))
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))


In [ ]:
for iris in [82, 83, 84, 85]:
    fig, ax = plt.subplots(1, figsize=(15,8))
    stage='sampleX'
    countbig = 0
    vspan = hlim[1]
    scan = int800.copy()
    scan = int800.copy()[scan.iris == iris]
    N = len(scan[stage].unique())
    shape = scan.data.values[0].shape
    print(shape)
    Z = np.zeros((N, shape[0],vspan[1]-vspan[0]))
    Y = np.zeros(N)
    count = 0
    for i in np.arange(N)[scan[stage].unique().argsort()]:
        Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()[:,vspan[0]:vspan[1]]
        Y[count] = scan[stage].unique()[i]
        count+=1

    ax.axes.set_prop_cycle(cycler(color = mcp.gen_color(cmap="plasma",n=N+1)))
    for i in range(Z.shape[0]):
        ax.plot(savgol_filter(Z.mean(axis=1)[i], 21, 3), label=Y[i])

    ax.set_title((h_color[1],'WP position', iris, ))
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))


In [ ]:
for iris in [82, 83, 84, 85]:
    fig, ax = plt.subplots(1, figsize=(15,8))
    stage='sampleX'
    countbig = 0
    vspan = hlim[2]
    scan = int800.copy()
    scan = int800.copy()[scan.iris == iris]
    N = len(scan[stage].unique())
    shape = scan.data.values[0].shape
    print(shape)
    Z = np.zeros((N, shape[0],vspan[1]-vspan[0]))
    Y = np.zeros(N)
    count = 0
    for i in np.arange(N)[scan[stage].unique().argsort()]:
        Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()[:,vspan[0]:vspan[1]]
        Y[count] = scan[stage].unique()[i]
        count+=1

    ax.axes.set_prop_cycle(cycler(color = mcp.gen_color(cmap="plasma",n=N+1)))
    for i in range(Z.shape[0]):
        ax.plot(savgol_filter(Z.mean(axis=1)[i], 21, 3) - min(savgol_filter(Z.mean(axis=1)[i], 21, 3)), label=Y[i])

    ax.set_title((h_color[2],'WP position', iris, ))
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))


In [ ]:
for iris in [82, 83, 84, 85]:
    fig, ax = plt.subplots(1, figsize=(15,8))
    stage='sampleX'
    countbig = 0
    vspan = hlim[3]
    scan = int800.copy()
    scan = int800.copy()[scan.iris == iris]
    N = len(scan[stage].unique())
    shape = scan.data.values[0].shape
    print(shape)
    Z = np.zeros((N, shape[0],vspan[1]-vspan[0]))
    Y = np.zeros(N)
    count = 0
    for i in np.arange(N)[scan[stage].unique().argsort()]:
        Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()[:,vspan[0]:vspan[1]]
        Y[count] = scan[stage].unique()[i]
        count+=1

    ax.axes.set_prop_cycle(cycler(color = mcp.gen_color(cmap="plasma",n=N+1)))
    for i in range(Z.shape[0]):
        ax.plot(savgol_filter(Z.mean(axis=1)[i], 21, 3) - min(savgol_filter(Z.mean(axis=1)[i], 21, 3)), label=Y[i])

    ax.set_title((h_color[3],'WP position', iris, ))
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))


In [ ]:
for iris in [82, 83, 84, 85]:
    fig, ax = plt.subplots(1, figsize=(15,8))
    stage='sampleX'
    countbig = 0
    vspan = hlim[4]
    scan = int800.copy()
    scan = int800.copy()[scan.iris == iris]
    N = len(scan[stage].unique())
    shape = scan.data.values[0].shape
    print(shape)
    Z = np.zeros((N, shape[0],vspan[1]-vspan[0]))
    Y = np.zeros(N)
    count = 0
    for i in np.arange(N)[scan[stage].unique().argsort()]:
        Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()[:,vspan[0]:vspan[1]]
        Y[count] = scan[stage].unique()[i]
        count+=1

    ax.axes.set_prop_cycle(cycler(color = mcp.gen_color(cmap="plasma",n=N+1)))
    for i in range(Z.shape[0]):
        ax.plot(savgol_filter(Z.mean(axis=1)[i], 21, 3) - min(savgol_filter(Z.mean(axis=1)[i], 21, 3)), label=Y[i])

    ax.set_title((h_color[4],'WP position', iris, ))
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))


In [ ]:
for iris in [82, 83, 84, 85]:
    fig, ax = plt.subplots(1, figsize=(15,8))
    stage='sampleX'
    countbig = 0
    vspan = hlim[5]
    scan = int800.copy()
    scan = int800.copy()[scan.iris == iris]
    N = len(scan[stage].unique())
    shape = scan.data.values[0].shape
    print(shape)
    Z = np.zeros((N, shape[0],vspan[1]-vspan[0]))
    Y = np.zeros(N)
    count = 0
    for i in np.arange(N)[scan[stage].unique().argsort()]:
        Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()[:,vspan[0]:vspan[1]]
        Y[count] = scan[stage].unique()[i]
        count+=1

    ax.axes.set_prop_cycle(cycler(color = mcp.gen_color(cmap="plasma",n=N+1)))
    for i in range(Z.shape[0]):
        ax.plot(savgol_filter(Z.mean(axis=1)[i], 21, 3) - min(savgol_filter(Z.mean(axis=1)[i], 21, 3)), label=Y[i])

    ax.set_title((h_color[5],'WP position', iris, ))
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))


In [ ]:
for iris in [82, 83, 84, 85]:
    fig, ax = plt.subplots(1, figsize=(15,8))
    stage='sampleX'
    countbig = 0
    vspan = hlim[6]
    scan = int800.copy()
    scan = int800.copy()[scan.iris == iris]
    N = len(scan[stage].unique())
    shape = scan.data.values[0].shape
    print(shape)
    Z = np.zeros((N, shape[0],vspan[1]-vspan[0]))
    Y = np.zeros(N)
    count = 0
    for i in np.arange(N)[scan[stage].unique().argsort()]:
        Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()[:,vspan[0]:vspan[1]]
        Y[count] = scan[stage].unique()[i]
        count+=1

    ax.axes.set_prop_cycle(cycler(color = mcp.gen_color(cmap="plasma",n=N+1)))
    for i in range(Z.shape[0]):
        ax.plot(savgol_filter(Z.mean(axis=1)[i], 21, 3) - min(savgol_filter(Z.mean(axis=1)[i], 21, 3)), label=Y[i])

    ax.set_title((h_color[6],'WP position', iris, ))
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))


### Tracing Thin Harmonics

In [ ]:
vlim = (250,300)
for iris in [82,83,84,85]:
    fig, ax = plt.subplots(1, figsize=(15,8))
    stage='sampleX'
    countbig = 0
    scan = int800.copy()
    scan = int800.copy()[scan.iris == iris]
    N = len(scan[stage].unique())
    shape = scan.data.values[0].shape
    print(shape)
    Z = np.zeros((N, vlim[1]-vlim[0], shape[1]))
    Y = np.zeros(N)
    count = 0
    for i in np.arange(N)[scan[stage].unique().argsort()]:
        Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()[vlim[0]:vlim[1],:]
        Y[count] = scan[stage].unique()[i]
        count+=1
    y = np.arange(len(Y)*(vlim[1]-vlim[0]))
    im = ax.imshow(Z.reshape(31*(vlim[1]-vlim[0]),shape[1]), aspect = 'auto', norm=LogNorm(vmin=100), cmap='ocean_r')
    ax.set_title(('WP position', iris))
    ax.axvspan(20,120, alpha=0.2, color='red')
    ax.axvspan(160,290, alpha=0.2, color='blue')
    ax.axvspan(450,550, alpha=0.2, color='white')
    ax.axvspan(550,650, alpha=0.2, color='yellow')
    ax.axvspan(800,950, alpha=0.2, color='orange')
    ax.axvspan(1020,1100, alpha=0.2, color='purple')
    ax.axvspan(1120,1200, alpha=0.2, color='black')

    ax.set_title(('WP position', iris))

    plt.colorbar(im, ax=ax)
    plt.xlim(0,1300)
    plt.show()
    
    

hlim = [(20,120), (160,290), (450,550), (550,650), (800,950), (1020,1100), (1120,1200)]
h_color = ['Red', 'Blue (5th?)', 'White', 'Yellow (7th?)', 'Orange (9th?)', 'Pink (11th?)', 'Black']

In [ ]:
vlim = (250,300)
for h_index in range(len(hlim)):
    print()
    print()
    print(h_color[h_index])
    for iris in [82, 83, 84, 85]:
        fig, ax = plt.subplots(1, figsize=(15,8))
        stage='sampleX'
        countbig = 0
        vspan = hlim[h_index]
        scan = int800.copy()
        scan = int800.copy()[scan.iris == iris]
        N = len(scan[stage].unique())
        shape = scan.data.values[0].shape
        print(shape)
        Z = np.zeros((N, vlim[1]-vlim[0],vspan[1]-vspan[0]))
        Y = np.zeros(N)
        count = 0
        for i in np.arange(N)[scan[stage].unique().argsort()]:
            Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()[vlim[0]:vlim[1],vspan[0]:vspan[1]]
            Y[count] = scan[stage].unique()[i]
            count+=1

        ax.axes.set_prop_cycle(cycler(color = mcp.gen_color(cmap="plasma",n=N+1)))
        for i in range(Z.shape[0]):
            ax.plot(savgol_filter(Z.mean(axis=1)[i], 21, 3) - min(savgol_filter(Z.mean(axis=1)[i], 21, 3)), label=Y[i])

        ax.set_title((h_color[h_index],'WP position', iris, ))
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    plt.show()


## Intensity Rotation Scan

In [ ]:
lowint = loading(save_folder_art, "Scan_220621_('MgO200',)um_800longnm_part2_rotationfocusscan_0to500_70000MCP.npz")
midint = loading(save_folder_art, "Scan_220621_('MgO200',)um_800longnm_part2_rotationfocusscan_500to1000_70000MCP.npz")
highint = loading(save_folder_art,"Scan_220621_('MgO200',)um_800longnm_part2_rotationfocusscan_1000to1210_70000MCP.npz")
int_rot = pd.concat([lowint, midint, highint], ignore_index=True)

In [ ]:
print('wp positions',int_rot['wp'].unique())
print('rotation degrees',int_rot['rotation'].unique())

In [ ]:
scan = int_rot.copy()
iris = int_rot['wp'].unique()[5]
fig, ax = plt.subplots(1, figsize=(15,8))
stage='rotation'
countbig = 0
scan = scan[scan.wp == iris]
N = len(scan[stage].unique())
shape = scan.data.values[0].shape
print(shape)
Z = np.zeros((N, shape[0], shape[1]))
Y = np.zeros(N)
count = 0
for i in np.arange(N)[scan[stage].unique().argsort()]:
    Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()
    Y[count] = scan[stage].unique()[i]
    count+=1
print(Z.shape)
y = np.arange(len(Y)*(shape[0]))
im = ax.imshow(Z.reshape(Z.shape[0]*(shape[0]),shape[1]), aspect = 'auto', norm=LogNorm(vmin=100,vmax=1000), cmap='ocean_r')
ax.set_title(('WP position', iris))
ax.set_yticks([])
#ax.set_yticklabels(Y)
ax.set_ylabel('Rotation over 360 deg')
plt.colorbar(im, ax=ax)
plt.xlim(0,1200)
plt.show()

In [ ]:
scan = int_rot.copy()
iris = int_rot['wp'].unique()[6]
fig, ax = plt.subplots(1, figsize=(15,8))
stage='rotation'
countbig = 0
scan = scan[scan.wp == iris]
N = len(scan[stage].unique())
shape = scan.data.values[0].shape
print(shape)
vlow = 190
vhigh = 350
Z = np.zeros((N, vhigh-vlow, shape[1]))
Y = np.zeros(N)
count = 0
for i in np.arange(N)[scan[stage].unique().argsort()]:
    Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()[vlow:vhigh]
    Y[count] = scan[stage].unique()[i]
    count+=1
print(Z.shape)
y = np.arange(len(Y)*(shape[0]))
im = ax.imshow(Z.reshape(Z.shape[0]*(Z.shape[1]),shape[1])[:Z.shape[0]*(vhigh-vlow)//4], aspect = 'auto', norm=LogNorm(vmin=100,vmax=2100), cmap='ocean_r')
ax.set_title(('WP position', iris))
ax.set_yticks([])
#ax.set_yticklabels(Y)
ax.set_ylabel('Rotation over 90 deg')
plt.colorbar(im, ax=ax)
plt.xlim(0,1200)
plt.savefig('zoom.png')
plt.show()

In [ ]:
vlim = [250,500,800,900,1000,1200,1600,2000,2200,2400, 2100]
for i in np.arange(len(int_rot['wp'].unique())):
    print(i)
    print(vlim[i])
    vmax = int(vlim[i])
    scan = int_rot.copy()
    iris = int_rot['wp'].unique()[i]
    fig, ax = plt.subplots(1, figsize=(15,8))
    stage='rotation'
    countbig = 0
    scan = scan[scan.wp == iris]
    N = len(scan[stage].unique())
    shape = scan.data.values[0].shape
    print(shape)
    vlow = 190
    vhigh = 350
    Z = np.zeros((N, vhigh-vlow, shape[1]))
    Y = np.zeros(N)
    count = 0
    for i in np.arange(N)[scan[stage].unique().argsort()]:
        Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()[vlow:vhigh]
        Y[count] = scan[stage].unique()[i]
        count+=1
    print(Z.shape)
    y = np.arange(len(Y)*(shape[0]))
    im = ax.imshow(Z.reshape(Z.shape[0]*(Z.shape[1]),shape[1])[:Z.shape[0]*(vhigh-vlow)//4], aspect = 'auto', norm=LogNorm(vmin=100,vmax=vmax), cmap='ocean_r')
    ax.set_title(('WP position', iris))
    ax.set_yticks([])
    #ax.set_yticklabels(Y)
    ax.set_ylabel('Rotation over 90 deg')
    plt.colorbar(im, ax=ax)
    plt.xlim(0,1200)
    plt.show()

In [ ]:
scan = int_rot.copy()
scan = scan[scan.wp == int_rot['wp'].unique()[5]]
fig, ax  = plt.subplots(1)
ev, deg, Z = plot_heatmap(scan, 'rotation', (3*1240/780,20), (190,350), 780, fig = fig, ax=ax, dp=10, norm = LogNorm(vmax=3e2))


In [ ]:
fig, ax = plt.subplots(1, figsize=(20,10), subplot_kw={'projection': 'polar'})


cmap = plt.get_cmap('BuPu')
r, theta = np.meshgrid(np.arange(len(ev)), np.radians(deg))
im = ax.contourf(theta,r, np.log10(Z), levels=50, cmap='ocean_r',vmin =2,vmax=3)




In [ ]:
vlim = (190,350)
vmax_list= [250,500,800,900,1000,1200,1600,2000,2200,2400, 2100]
hlim = [(0,130), (90,300), (380,480), (500,670), (720,960), (920,1100), (1120,1200)]
h_color = ['red', 'orange' , 'yellow', 'green', 'blue', 'indigo', 'violet']
for i in range(len(int_rot.wp.unique()))[5:10:2]:
    iris = int_rot.wp.unique()[i]
    vmax = vmax_list[i]
    fig, ax = plt.subplots(1, figsize=(15,8))
    stage='rotation'
    countbig = 0
    scan = int_rot.copy()
    scan = int_rot.copy()[scan.wp == iris]
    N = len(scan[stage].unique())
    shape = scan.data.values[0].shape
    print(shape)
    Z = np.zeros((N, vlim[1]-vlim[0], shape[1]))
    Y = np.zeros(N)
    count = 0
    for i in np.arange(N)[scan[stage].unique().argsort()]:
        Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()[vlim[0]:vlim[1],:]
        Y[count] = scan[stage].unique()[i]
        count+=1
    y = np.arange(len(Y)*(vlim[1]-vlim[0]))
    im = ax.imshow(Z.reshape(Z.shape[0]*(vlim[1]-vlim[0]),Z.shape[2]), aspect = 'auto', norm=LogNorm(vmin=100, vmax=vmax), cmap='ocean_r')
    ax.set_title(('WP position', iris))
    for h in range(len(hlim)):
        ax.axvspan(hlim[h][0],hlim[h][1], alpha=0.2, color=h_color[h])


    ax.set_title(('WP position', iris))

    plt.colorbar(im, ax=ax)
    plt.show()
    
    


## Short Pulse

In [ ]:
lowint = loading(save_folder_art, "Scan_220621_('MgO200',)um_800longnm_part2_intfocus_short_0to300_70000MCP.npz")
highint = loading(save_folder_art,"Scan_220621_('MgO200',)um_800longnm_part2_intfocus_short_300toend_70000MCP.npz")
both = pd.concat([lowint, highint], ignore_index=True)

In [ ]:
both.wp.unique()

In [ ]:
normlim = [(100,200), (100,200), (100,300), (100,400), (100, 500), (100,500), (100,500)]
count_lim = 0
lowlim = 200
highlim = 400
for iris in both.wp.unique():
    fig, ax = plt.subplots(1, figsize=(15,8))
    vmin = normlim[count_lim][0]
    vmax = normlim[count_lim][1]
    count_lim += 1
    stage='rotation'
    countbig = 0
    scan = both.copy()
    scan = both.copy()[scan.wp == iris]
    N = len(scan[stage].unique())
    shape = scan.data.values[0].shape
    print(shape)
    Z = np.zeros((N, highlim-lowlim, shape[1]))
    Y = np.zeros(N)
    count = 0
    for i in np.arange(N)[scan[stage].unique().argsort()]:
        Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()[lowlim:highlim,:]
        Y[count] = scan[stage].unique()[i]
        count+=1
    print(Z.shape)
    y = np.arange(len(Y)*(shape[0]))
    im = ax.pcolormesh(Z.reshape(Z.shape[0]*(highlim-lowlim),Z.shape[2]), norm=LogNorm(vmin=100,vmax=500), cmap='ocean_r')
    ax.set_title(('WP position', iris))
    ax.set_yticks([])
    ax.set_ylabel('Rotation over 180')
    plt.colorbar(im, ax=ax)
    plt.show()

# Argon Polarisation Test


In [ ]:
ar = loading(save_folder_art, "Scan_220621_('Ar',)um_800nm_part2_polarisation_30000MCP.npz")

In [ ]:
print('wp positions',ar['wp'].unique())
print('rotation degrees',ar['wp2'].unique())

In [ ]:
ar.head()

In [ ]:
vlim = [250,500,800,900,1000,1200,1600,2000,2200,2400, 2100]
for i in np.arange(len(ar['wp'].unique()))[-2:]:
    vmax = int(vlim[i])
    vmax = 10000
    scan = ar.copy()
    iris = ar['wp'].unique()[i]
    print(iris)
    fig, ax = plt.subplots(1, figsize=(15,8))
    stage='wp2'
    countbig = 0
    scan = scan[scan.wp == iris]
    N = len(scan[stage].unique())
    shape = scan.data.values[0].shape
    print(shape)
    vlow = 275
    vhigh = 355
    shape = (vhigh-vlow, shape[1])
    Z = np.zeros((N, shape[0], shape[1]))
    Y = np.zeros(N)
    count = 0

    for i in np.arange(N)[scan[stage].unique().argsort()]:
        Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()[vlow:vhigh]
        Y[count] = scan[stage].unique()[i]
        count+=1
    print(Z.shape)
    y = np.arange(len(Y)*(Z.shape[1]))
    im = ax.pcolormesh(Z.reshape(Z.shape[0]*(Z.shape[1]),Z.shape[2]), norm=LogNorm(vmin=100,vmax=vmax), cmap='ocean_r')
    #ax.set_title(('WP position', iris))
    #ax.set_yticks(y[Z.shape[1]//2::Z.shape[1]])
    #ax.set_yticklabels(Y)
    #plt.colorbar(im, ax=ax)
    plt.show()

In [ ]:
print(Y)

In [ ]:
plt.imshow(ar['data'][100], vmin=100, vmax=2000, cmap='ocean_r')
plt.ylim(, 355)

In [ ]:
vlim = [250,500,800,900,1000,1200,1600,2000,2200,2400, 2100]
for i in np.arange(len(ar['wp'].unique()))[-2:]:
    vmax = int(vlim[i])
    vmax = 10000
    scan = ar.copy()
    iris = ar['wp'].unique()[i]
    print(iris)
    fig, ax = plt.subplots(1, figsize=(15,8))
    stage='wp2'
    countbig = 0
    scan = scan[scan.wp == iris]
    N = len(scan[stage].unique())
    shape = scan.data.values[0].shape
    print(shape)
    vlow = 275
    vhigh = 355
    shape = (vhigh-vlow, shape[1])
    Z = np.zeros((N, shape[1]))
    Y = np.zeros(N)
    count = 0
    for i in np.arange(N)[scan[stage].unique().argsort()]:
        Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()[vlow:vhigh].sum(axis=0)
        Y[count] = scan[stage].unique()[i]
        count+=1
    im = ax.pcolormesh(np.arange(shape[1]), Y, Z, norm=LogNorm(), cmap='ocean_r')
    
    line_list = [45, 140, 230, 275, 459, 879,1055, 1179, 1275, 1345]
    for line in line_list:
        ax.axvline(line, color='red')
    
    plt.show()

In [ ]:
vlim = [250,500,800,900,1000,1200,1600,2000,2200,2400, 2100]
Z_array = {}
for i in np.arange(len(ar['wp'].unique())):
    vmax = int(vlim[i])
    vmax = 10000
    scan = ar.copy()
    iris = ar['wp'].unique()[i]
    print(iris)
    stage='wp2'
    countbig = 0
    scan = scan[scan.wp == iris]
    N = len(scan[stage].unique())
    shape = scan.data.values[0].shape
    print(shape)
    vlow = 275
    vhigh = 355
    shape = (vhigh-vlow, shape[1])
    Z = np.zeros((N, shape[1]))
    Y = np.zeros(N)
    count = 0
    for i in np.arange(N)[scan[stage].unique().argsort()]:
        Z[count] = scan[scan[stage]==scan[stage].unique()[i]].data.mean()[vlow:vhigh].sum(axis=0)
        Y[count] = scan[stage].unique()[i]
        count+=1
    Z_array[iris] = [Y, Z]

In [ ]:
Z_array

In [ ]:
test[1].shape

In [ ]:
for wp_iris in list(Z_array.keys()):
    fig, ax = plt.subplots(1)
    ax.axes.set_prop_cycle(cycler(color = mcp.gen_color(cmap="plasma",n=len(line_list)+1)))
    test = Z_array[wp_iris]
    for line in line_list:
        plt.plot(test[0], savgol_filter(test[1][:,line]/max(test[1][:,line]), 11, 3), label = 'pix number {}'.format(line))

    ax.set_title('Waveplate 1 : {}'.format(wp_iris))
    plt.xlabel('Polariser Angle')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
Z_array[list(Z_array.keys())[0]]

In [ ]:
wp_iris = list(Z_array.keys())[-1]
fig, ax = plt.subplots(1)
ax.axes.set_prop_cycle(cycler(color = mcp.gen_color(cmap="plasma",n=len(line_list)+1)))
test = Z_array[wp_iris]
for line in line_list:
    peaks, _ = find_peaks(test[1][:,line]/max(test[1][:,line]), )
    plt.plot(test[0], savgol_filter(test[1][:,line]/max(test[1][:,line]), 11, 3), label = 'pix number {}'.format(line))

ax.set_title('Waveplate 1 : {}'.format(wp_iris))
plt.xlabel('Polariser Angle')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Polarisation Scan 

In [ ]:
pol = loading(save_folder_art, "Scan_220621_('MgO',)um_800nm_part2_polar_rotation_comparison_course_30000MCP.npz")

In [ ]:
plt.imshow(pol['data'][301], cmap='ocean_r', norm = LogNorm(vmax=300))
plt.title('Polarisation {}, Rotation {}'.format(pol['wp2'][301],pol['rotation'][301]))
plt.axhline(200)
plt.axhline(400)
plt.ylim()

In [ ]:
plt.imshow(pol['data'][501], cmap='ocean_r', norm = LogNorm(vmax=300))
plt.title('Polarisation {}, Rotation {}'.format(pol['wp2'][501],pol['rotation'][501]))
plt.axhline(100)
plt.axhline(300)
plt.ylim()

In [ ]:
pol['rotation'].unique()
check['data'].values[0]

In [ ]:
check

In [ ]:
check = pol[pol['rotation']==90]
idx=0
plt.imshow(check['data'].values[idx], cmap='ocean_r', norm = LogNorm(vmax=300))
plt.title('Polarisation {}, Rotation {}'.format(check['wp2'].values[idx],check['rotation'].values[idx]))
plt.axhline(100)
plt.axhline(480)
plt.ylim()

In [ ]:
for p in pol['wp2'].unique():
    forscan = pol[pol['wp2']==p]
    forscan = forscan[forscan['wp']==60]
    fig, ax = plt.subplots(2,3, figsize = (10,4))
    fig.suptitle("Polarisation {}".format(p), fontsize=16)
    ax = ax.flatten()
    count = 0
    for rot in [30,60,90,120,150,180]:
        ax[count].imshow(forscan[forscan['rotation']==rot]['data'].values[0],cmap='ocean_r', norm = LogNorm(vmax=300))
        peaks, _ = find_peaks(forscan[forscan['rotation']==rot]['data'].values[0].mean(axis=0), width = 20)
        ax[count].axhline(np.argmax(forscan[forscan['rotation']==rot]['data'].values[0][:,peaks].mean(axis=1)), color='red')
        ax[count].set_xlabel('Rotation {}'.format(rot, fontsize=14))
        count +=1
    
    plt.tight_layout()

In [ ]:
plt.plot(savgol_filter(forscan[forscan['rotation']==90]['data'].values[0].mean(axis=0), 11,3))
peaks, _ = find_peaks(savgol_filter(forscan[forscan['rotation']==90]['data'].values[0].mean(axis=0),11,3), width = 20)
plt.plot(peaks, forscan[forscan['rotation']==90]['data'].values[0].mean(axis=0)[peaks], 'rx')
plt.show()

plt.imshow(forscan[forscan['rotation']==90]['data'].values[0],cmap='ocean_r', norm = LogNorm(vmax=300))
plt.axhline(np.argmax(forscan[forscan['rotation']==90]['data'].values[0][:,peaks].mean(axis=1)), color='red')



In [ ]:
forscan = pol[pol['wp2']==p]
forscan = forscan[forscan['wp']==60]
fig, ax = plt.subplots(2,3, figsize = (10,4))
fig.suptitle("Polarisation {}".format(p), fontsize=16)
ax = ax.flatten()
count = 0
for rot in [30,60,90,120,150,180]:
    ax[count].plot(forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0)- forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0)[1300:1500].min())
    peaks, _ = find_peaks(forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0), width = 20)
    #ax[count].plot(peaks,forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0)[peaks],'rx')
    ax[count].set_xlabel('Rotation {}'.format(rot, fontsize=14))
    count +=1


In [ ]:
forscan = pol[pol['wp2']==p]
forscan = forscan[forscan['wp']==60]
fig, ax = plt.subplots(1)
ax.axes.set_prop_cycle(cycler(color = mcp.gen_color(cmap="plasma",n=len(forscan['rotation'].unique()[::3])+1)))
for rot in forscan['rotation'].unique()[::3]:
    plt.plot(forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0), label = rot)
    peaks, _ = find_peaks(forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0), width = 20)
    plt.plot(peaks,forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0)[peaks],'rx')

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
p

In [ ]:
forscan = pol[pol['wp2']==p]
forscan = forscan[forscan['wp']==60]
fig, ax = plt.subplots(1, figsize = (10,4))
fig.suptitle("Polarisation {}".format(p), fontsize=16)
Z = np.zeros((len(forscan['rotation'].unique()),forscan['data'].values[0].shape[1]))
count = 0
peak_list = []
rot_ax = np.zeros(len(forscan['rotation'].unique()))
for rot in forscan['rotation'].unique():
    Z[count] = forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0) - forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0)[1300:1500].min()
    rot_ax[count]=rot
    peaks, _ = find_peaks(forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0), width = 20)
    peak_list.append(peaks)
    count +=1
ax.contourf(np.arange(Z.shape[1]), rot_ax, Z, levels= 100)

In [ ]:

for p in pol['wp2'].unique():
    forscan = pol[pol['wp2']==p]
    forscan = forscan[forscan['wp']==50]
    fig, ax = plt.subplots(1, figsize = (10,4))
    fig.suptitle("Polarisation {}".format(p), fontsize=16)
    Z = np.zeros((len(forscan['rotation'].unique()),forscan['data'].values[0].shape[1]))
    count = 0
    peak_list = []
    rot_ax = np.zeros(len(forscan['rotation'].unique()))
    for rot in forscan['rotation'].unique():
        Z[count] = forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0) - forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0)[1300:1500].min()
        rot_ax[count]=rot
        peaks, _ = find_peaks(forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0), width = 20)
        peak_list.append(peaks)
        count +=1
    im = ax.contourf(np.arange(Z.shape[1]), rot_ax, Z, cmap='ocean_r', vmin = 100, vmax=50000, levels=100)
    plt.colorbar(im)
    plt.ylabel('Rotation')
    plt.tight_layout()

In [ ]:
p=110
forscan = pol[pol['wp2']==p]
forscan = forscan[forscan['wp']==60]
fig, ax = plt.subplots(1, figsize = (10,4))
fig.suptitle("Polarisation {}".format(p), fontsize=16)
Z = np.zeros((len(forscan['rotation'].unique()),forscan['data'].values[0].shape[1]))
count = 0
peak_list = []
rot_ax = np.zeros(len(forscan['rotation'].unique()))
for rot in forscan['rotation'].unique():
    Z[count] = forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0) - forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0)[1300:1500].min()
    rot_ax[count]=rot
    peaks, _ = find_peaks(forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0), width = 20)
    peak_list.append(peaks)
    count +=1
im = ax.contourf(np.arange(Z.shape[1]), rot_ax, Z, cmap='ocean_r', vmin = 100, vmax=50000, levels=100)
for loc in [220, 385,752,1035]:
    ax.axvspan(loc-20, loc+10, color='red', alpha=0.2)
plt.colorbar(im)
plt.ylabel('Rotation')
plt.tight_layout()

In [ ]:
peak_loc = [220, 385,752,1035]
loc_low = -20
loc_high = 10

In [ ]:
Z.shape

In [ ]:
p=40
forscan = pol[pol['wp2']==p]
forscan = forscan[forscan['wp']==60]
fig, ax = plt.subplots(1, figsize = (10,10),subplot_kw={'projection': 'polar'})
fig.suptitle("Polarisation {}".format(p), fontsize=16)
Z = np.zeros((len(forscan['rotation'].unique()),forscan['data'].values[0].shape[1]))
count = 0
peak_list = []
rot_ax = np.zeros(len(forscan['rotation'].unique()))
for rot in forscan['rotation'].unique():
    Z[count] = forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0) - forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0)[1300:1500].min()
    rot_ax[count]=rot
    peaks, _ = find_peaks(forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0), width = 20)
    peak_list.append(peaks)
    count +=1
for loc in peak_loc:
    plt.plot(np.radians(rot_ax), np.log10(Z[:,loc+loc_low:loc+loc_high].mean(axis=1)/min(Z[:,loc+loc_low:loc+loc_high].mean(axis=1))))

plt.tight_layout()

In [ ]:
for p in pol['wp2'].unique():
    forscan = pol[pol['wp2']==p]
    forscan = forscan[forscan['wp']==60]
    fig, ax = plt.subplots(1, figsize = (5,10),subplot_kw={'projection': 'polar'})
    Z = np.zeros((len(forscan['rotation'].unique()),forscan['data'].values[0].shape[1]))
    count = 0
    peak_list = []
    rot_ax = np.zeros(len(forscan['rotation'].unique()))
    for rot in forscan['rotation'].unique():
        Z[count] = forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0) - forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0)[1300:1500].min()
        rot_ax[count]=rot
        peaks, _ = find_peaks(forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0), width = 20)
        peak_list.append(peaks)
        count +=1
    for loc in peak_loc:
        ax.plot(np.radians(rot_ax), np.log10(Z[:,loc+loc_low:loc+loc_high].mean(axis=1)))
    ax.set_thetalim(0,np.radians(180))
    ax.set_rlim(2.5)
    plt.title('Polarisation {}'.format(p))
    plt.legend(['1st', '2nd', '3rd', '4th'])
    plt.tight_layout()

In [ ]:
har1 = {}
har2 = {}
har3 = {}
har4 = {}
for p in pol['wp2'].unique():
    forscan = pol[pol['wp2']==p]
    forscan = forscan[forscan['wp']==60]
    Z = np.zeros((len(forscan['rotation'].unique()),forscan['data'].values[0].shape[1]))
    count = 0
    peak_list = []
    rot_ax = np.zeros(len(forscan['rotation'].unique()))
    for rot in forscan['rotation'].unique():
        Z[count] = forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0) - forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0)[1300:1500].min()
        rot_ax[count]=rot
        peaks, _ = find_peaks(forscan[forscan['rotation']==rot]['data'].values[0].sum(axis=0), width = 20)
        peak_list.append(peaks)
        count +=1
    
    har1[p] = [np.radians(rot_ax), Z[:,peak_loc[0]+loc_low:peak_loc[0]+loc_high].mean(axis=1)]
    har2[p] = [np.radians(rot_ax), Z[:,peak_loc[1]+loc_low:peak_loc[1]+loc_high].mean(axis=1)]
    har3[p] = [np.radians(rot_ax), Z[:,peak_loc[2]+loc_low:peak_loc[2]+loc_high].mean(axis=1)]
    har4[p] = [np.radians(rot_ax), Z[:,peak_loc[3]+loc_low:peak_loc[3]+loc_high].mean(axis=1)]
                                                                  

In [ ]:
fig, ax = plt.subplots(2,2, figsize = (10,10),subplot_kw={'projection': 'polar'})
ax = ax.flatten()

har_list = [ har3, har4,har1, har2]
h_name = ['3rd','4th','1st', '2nd',]
for h in range(4):
    ax[h].set_title(h_name[h])
    ax[h].axes.set_prop_cycle(cycler(color = mcp.gen_color(cmap="plasma",n=19)[::2]))
    dic = har_list[h]
    for polar in list(dic.keys()):
        ax[h].plot(dic[polar][0][1:-1], savgol_filter(dic[polar][1][1:-1]/max(dic[polar][1][1:-1]),9,3), label = polar)
    ax[h].set_thetalim(np.radians(6), np.radians(172))
    ax[h].set_xticks(np.arange(np.radians(6), np.radians(172),np.radians(6)),endpoint=False)
    ax[h].set_xticklabels([])
    ax[h].set_rlim(min(savgol_filter(dic[polar][1][1:-1]/max(dic[polar][1][1:-1]),9,3))-0.1)
    ax[h].legend(fontsize=10)

plt.tight_layout()

In [ ]:
plt.plot(np.linspace(30,110,9), [104,72,54,43,40,48,48,36+45,18+90])
plt.xlabel('polarisation')
plt.ylabel('Angle of the peak')